# Combate à Poluição Plástica nos Oceanos e Praias: Um Desafio de Machine Learning

## Global Solution - 3ECR

##Integrantes:
- Bianca Fioretti dos Santos (RM96019)
- Gustavo Kenzo Abe (RM 93944)
- Greidimar Ferreira Lagares (RM 93827)



### Descrição

O dataset intitulado "Global Microplastic Initiative" (https://www.adventurescientists.org/microplastics.html) desenvolvido pela organização Adventure Scientists (https://www.adventurescientists.org/) é uma pesquisa com um dos maiores e mais diversos conjuntos de dados globais sobre poluição por microplásticos até o momento. De 2013 a 2017, a Adventure Scientists mobilizou milhares de voluntários treinados para ajudar a identificar a extensão da poluição por microplásticos em sistemas marinhos e de água doce em todo o mundo. Estes dados estão a ser utilizados por empresas, governos e indivíduos para limitar o desperdício de plástico e estudar seus efeitos no meio ambiente.

Com este dataset podemos:
- **Previsão da abundância de resíduos plásticos:** Desenvolver um modelo que preveja a abundância de resíduos plásticos em um determinado local com base em fatores ambientais como temperatura da água, salinidade e proximidade de assentamentos humanos.

- **Identificação de fontes de poluição plástica:** Criar um modelo que identifique as prováveis fontes de poluição plástica em uma determinada área, considerando fatores como práticas de gestão de resíduos, atividades industriais e turismo costeiro.

### Metodologia utilizada
**Classificação**
Escolhemos a classificação pois queremos prever a classe/categoria das analises coletadas de acordo com o local em que foram recolhidas pelos voluntarios. Acreditamos que esses dados serão de grande ajuda no projeto PlasticPurge uma vez que com estes dados podemos verificar quais locais possuem maior quantidade de microplásticos e utilizar este machine learning para decidir onde nossa embarcação deve atuar com mais urgência.

Em nosso arquivo será utilizado o Algoritmo Random Forest e o Algoritmo KNN.


In [19]:
#  Importação de bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Algoritmo Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Criação do dataframe

Agora iremos criar um dataframe chamado "df" a partir dos dados do arquivo "Microplastic_Data_AdventureScientists.csv" anexado pelo grupo junto a entrega do código fonte. Este dataset pode ser encontrado no site https://www.adventurescientists.org/microplastics.html.

In [20]:
# Criação do dataframe
df = pd.read_csv('Microplastic_Data_AdventureScientists.csv')

# Informações dos atributos (formato dos dados)
df.info()

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393 entries, 0 to 1392
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Sample Date      1389 non-null   object 
 1   Latitude         1393 non-null   float64
 2   Longitude        1393 non-null   float64
 3   Total Pieces/L   1393 non-null   int64  
 4   Unnamed: 4       0 non-null      float64
 5   Unnamed: 5       0 non-null      float64
 6   Unnamed: 6       0 non-null      float64
dtypes: float64(5), int64(1), object(1)
memory usage: 76.3+ KB


In [21]:
# Estatísticas descritivas dos atributos
df.describe()

,Latitude,Longitude,Total Pieces/L,Unnamed: 4,Unnamed: 5,Unnamed: 6
count,1393.000000,1393.000000,1393.000000,0.0,0.0,0.0
mean,17.360050,-50.711904,12.750179,NaN,NaN,NaN
std,26.967994,66.246497,27.319984,NaN,NaN,NaN
min,-65.177500,-169.926130,0.000000,NaN,NaN,NaN
25%,14.302333,-83.154150,1.000000,NaN,NaN,NaN
50%,18.704733,-56.433333,4.000000,NaN,NaN,NaN
75%,29.467000,-28.020000,12.000000,NaN,NaN,NaN
max,81.090000,174.135000,323.000000,NaN,NaN,NaN


In [22]:
# Excluindo colunas vazias
df = df[['Sample Date', 'Latitude', 'Longitude', 'Total Pieces/L ']]
df.columns

Index(['Sample Date', 'Latitude', 'Longitude', 'Total Pieces/L '], dtype='object')

# Vizualização das informações do dataframe

Agora iremos gerar gráficos e mapas para ter noção das localizações mais críticas de forma visual antes de partimos para nossas analíses preditivas. Aqui buscamos atender ao requisito "Identificação de fontes de poluição plástica", pois com os gráficos buscamos destacar os locais com maior quantidade de poluição dos oceanos.

In [ ]:
# Vizualização da distribuição de microplásticos por localização
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='Longitude', y='Latitude', hue='Total Pieces/L ', palette='viridis')
plt.title('Distribuição de Microplásticos por Localização')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Gerando mapas com os valores encontrados
mapa = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=2)

for index, row in df.iterrows():
    quantidade = row['Total Pieces/L ']
    cor = 'red' if quantidade > 100 else 'orange' if quantidade > 50 else 'yellow' if quantidade > 10 else 'green'
    folium.CircleMarker(location=[row['Latitude'], row['Longitude']], radius=quantidade/10, color=cor, fill=True, fill_color=cor).add_to(mapa)

mapa

# Criação e separação dos dados de teste

Agora para atender ao requisito "Previsão da abundância de resíduos plásticos", vamos realizar a nossa analise preditiva para saber onde nossa embarcação precisa atuar com mais urgência de acordo com os resultados.

In [23]:
# Criação dos dados
X = df[['Latitude', 'Longitude']]
y = df['Total Pieces/L ']

X.head()

,Latitude,Longitude
0,59.378683,-153.520867
1,59.368133,-153.552333
2,59.341467,-153.549767
3,59.638717,-153.395667
4,59.373600,-153.448067


In [24]:
# Separação de dados
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

print(f'X_treino: {X_treino.shape}')
print(f'X_teste: {X_teste.shape}')
print(f'y_treino: {y_treino.shape}')
print(f'y_teste: {y_teste.shape}')

X_treino: (975, 2)
X_teste: (418, 2)
y_treino: (975,)
y_teste: (418,)


In [25]:
# Treinamento do modelo Random Forest
modelo1 = RandomForestClassifier(random_state=42)
modelo1.fit(X_treino, y_treino)

# Gerar analise preditiva
y_preditivo_RF = modelo1.predict(X_teste)
y_preditivo_RF

array([  7,   0, 116,  28,   3,   2,   0,   4,   9,   0,  59,  10,   4,
        21,   2,   2,   0,   3, 114,   1,  22,   0,  10,   6,   0,   3,
         0,   0,  15,   7,  68,   0,   0,   1,   4,   2,   1,   3,   3,
        12,   3,  38, 116,   2,  15,  13,  54,  16,   0,   8,   1, 239,
         0,  11,  29,   3,   5,   0,   2,   2,  11,   0,   4,  10,   4,
         1,  12,  90,   1,   3,   2,   0,   0,   3,  50,   3,   4,   3,
         2,  41,   0,   0,   0,  62,   3,   3,  32,   2,   0,   5,   5,
        16,  13,  14,  45,   6,   6,   0,   3,   1,   4,   1,   3,   0,
         1,   2,   0,  19,   0,   0,   3,   2,  38,  10,   2,   3,   1,
         8,   1,   4,  12,   1,   0,   5,   6,   0,   6,  45,   1,   3,
         0,   2,  22,   4,   4,   2,  12,   0,   0,   8,  23,   4,   3,
         2,  26,  21,   4,  17,   0,   4,   4,   3,   0,   2,   3,  12,
        13,  20,   3,   1,  10,   9,  61,   3,  13,   1,   4,   0,   1,
        14,   8,   3,  12,   5,   6,   8,   4,  10,   0,   0,   

In [26]:
# Utilização KNN para treinamento
modelo2 = KNeighborsClassifier()
modelo2.fit(X_treino, y_treino)

# Gerar analise preditiva
y_preditivo_KNN = modelo2.predict(X_teste)
y_preditivo_KNN

array([ 0,  0,  7, 10,  3,  2,  0,  4,  4,  0,  5,  3,  1,  3,  2,  2,  6,
        0,  0,  0,  5,  0,  3,  6,  0,  3,  0,  0,  3,  0,  0,  0,  0,  1,
        4,  0,  1,  3,  2, 11, 10,  3,  7,  1,  0, 11,  2,  2,  0,  1,  1,
       15,  0,  5,  2,  3,  3,  0,  2,  7,  1,  0,  0,  2,  0,  1,  4,  0,
        1,  1,  2,  0,  0,  2,  0,  3,  4,  3,  2,  3,  0,  0,  0,  2,  0,
        3, 10,  0,  4,  2,  0, 16,  5,  3,  4,  0,  3,  0, 14,  0,  3,  0,
        0,  0,  1,  2,  0,  3,  0,  0,  3,  7,  0,  2,  0,  9,  1,  0,  1,
        0,  6,  1,  0,  4,  2,  0, 12,  5,  1,  1,  1,  1,  4, 12,  3,  2,
        1,  0,  0,  0,  2,  0,  0,  0,  0, 68,  1, 11, 10,  0,  0,  0,  0,
        8,  2,  2,  1,  4,  0,  0,  7,  0,  6,  0,  0,  0,  0,  0,  1,  1,
        6,  0,  6,  5,  6,  3,  4,  1,  0,  6,  2,  0,  0,  4,  0,  2,  8,
        1,  2,  3,  5,  1,  0,  5,  0,  4,  3,  0,  0,  6, 16,  0,  1,  9,
        0,  0,  0,  2,  1,  1,  1,  0,  5,  3,  1, 10,  2,  0,  7,  0,  0,
        0,  0,  0,  7,  0

# Resultados

A seguir, vamos realizar o cálculo para saber a acuracia dos modelos criados e calcular qual possue maior eficácia na predição dos eventos.

In [27]:
# Avaliação do modelo Random Forest
acuracia1 = 100 *  (accuracy_score(y_teste, y_preditivo_RF))
print(f'A acurácia do modelo Random Forest é: {acuracia1:.2f}%')

A acurácia do modelo Random Forest é: 15.31%


In [28]:
# Avaliação do modelo KNN
acuracia2 = 100 *  (accuracy_score(y_teste, y_preditivo_KNN))
print(f'A acurácia do modelo KNN é: {acuracia2:.2f}%')

A acurácia do modelo KNN é: 18.18%


# Conclusões

Podemos observar que ambos os algoritmos utilizados desempenharam uma baixa acurácia mesmo utilizando dois algoritmos muito eficazes para nosso objetivo. Isso se dá ao fato de nosso dataset possuir poucas variáveis.
A inclusão de mais variáveis preditivas além da latitude e longitude utilizados, como temperatura da água, salinidade, distâncias até a costa ou até cidades próximas, densidade populacional, etc, podem aumentar a precisão do modelo o que o tornaria mais confiavel para a utilização em nosso projeto PlasticPurge.